<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import pdb
import requests
import time
import random
import os
import glob

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed

In [8]:
def extract_news_from_page(page):
    html = requests.get(page).text
    soup = BeautifulSoup(html)

    links = soup.find_all("a",
                          {"class": "feed-post-link"},
                          href=True)
    links = [link["href"] for link in links]

    return links


def get_full_html_from_news(news_link, driver):
    
    driver.get(news_link)
    ### Scroll para carregar o restante da página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    ### Aguardando carregar
    time.sleep(10)
    ## Clicando no botão de carregar mais comentários
#     more_comments = True
#     while more_comments:
#         try:
#             driver.find_element_by_class_name("glbComentarios-botao-mais").click()
#         except:
#             more_comments = False

    return driver.page_source

def get_title(soup):
    try:
        title = soup.find_all("h1", {"class": "content-head__title"})[0].text
    except:
        try:
            title = soup.find_all("h1", {"class": "entry-title"})[0].text            
        except:
            title = None 
        
    return title

def content_sportv(soup):
    content = soup.find_all("div",
                                {"class": "materia-conteudo entry-content"})
            
    paragraphs = content[0].findChildren("p", recursive=True) 
    
    return paragraphs

def extract_text_from_news_link(news_link, driver):
    html = get_full_html_from_news(news_link, driver)
    soup = BeautifulSoup(html)
    
    try:
        date = soup.find_all("time")[0].text
    except:
        date = None
        
    title = get_title(soup) 
        
    #print("Getting {}, from {}".format(title, date))
    paragraphs = soup.find_all("p",
                                {"class": "content-text__container "})
    
    if len(paragraphs) == 0:
        content = soup.find_all("div",
                                {"class": "corpo-conteudo"})
        if len(content) == 0:
            content = soup.find_all("div",
                                {"class": "materia-conteudo entry-content"})
            
        paragraphs = content[0].findChildren("p", recursive=True) 
    
    paragraphs = [paragraph.text for paragraph in paragraphs]
    
    ### Filter date from first p element
    if date in paragraphs[0]:
        paragraphs = paragraphs[1:]
        
    article_text = "".join(paragraphs)
    data = pd.DataFrame([[date, title, article_text, news_link]],
                        columns=["date", "title", "article_text", "article_link"])
    return data

def get_all_files_in_a_folder(team, full=False):
    path = "data/{}".format(team)
    
    files = [f for f in glob.glob(path + "**/*.csv", recursive=True)]
    
    if not full:
        files = [file.replace(path + "/", "") for file in files]
        files = [file.replace(".csv", "") for file in files]
    
    return files

def aggregate_all_saved_sections_from_zones_files(team):
    files = get_all_files_in_a_folder(team, full=True)
    
    all_dataframes = []
    for file in files:
        all_dataframes.append(pd.read_csv(file))
        
    return pd.concat(all_dataframes)


def news_from_soccer_club(team, n_pages=2):
    page_prefix = "https://globoesporte.globo.com/futebol/times/{}/index/feed/pagina-{}.ghtml"
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
    driver.implicitly_wait(5)

    try:
        data = pd.read_csv("data/{}.csv".format(team), index_col=False)
        downloaded = list(data["article_link"].values)  
        data.to_csv("data/backup-{}.csv".format(team), index=False)
    except:
        downloaded = []
        data = pd.DataFrame()
        
    news_links = []
    for n_page in tqdm(range(1, n_pages + 1)):
        page = page_prefix.format(team, n_page)
        new_news_links = extract_news_from_page(page)
        news_links += new_news_links

    news_links = [link for link in news_links if "/noticia/" in link]
    
    #news_links = random.shuffle(news_links)
    news_links = [news_link for news_link in news_links if news_link not in downloaded]

    print("{} links to extract".format(len(news_links)))
    
    for i, news_link in enumerate(tqdm(news_links)):
        try: 
            news_data = extract_text_from_news_link(news_link, driver)
            data = pd.concat([data, news_data], sort=True)
        except:
            print("Couldn't get from {}".format(news_link))
        if i % 5 == 0:
            #print("Saving {} news".format(len(data)))
            data.to_csv("data/{}.csv".format(team), index=False)

    try:
        data["article_time"] = data["date"].apply(lambda x: x.strip().split(" ")[1] if not pd.isnull(x) else x)
        data["article_date"] = data["date"].apply(lambda x: x.strip().split(" ")[0] if not pd.isnull(x) else x)
        data.to_csv("data/{}.csv".format(team), index=False)
    except:
        print("no date")

    driver.close()
    return data

def prepare_link_to_filename(link):
    link = link.replace("https://", "")
    link = link.replace("/", "-")
    link = link.replace(".", "-")    
    
    return link

def news_from_soccer_club_single(team, start_page, end_page):
    page_prefix = "https://globoesporte.globo.com/futebol/times/{}/index/feed/pagina-{}.ghtml"
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
    driver.implicitly_wait(5)

    directory = "data/{}".format(team)
    
    if not os.path.isdir(directory):
        os.makedirs(directory, exist_ok=True)
        
    data = pd.DataFrame()
        
    downloaded = get_all_files_in_a_folder(team)
    
    news_links = []
    for n_page in tqdm(range(start_page, end_page + 1)):
        page = page_prefix.format(team, n_page)
        new_news_links = extract_news_from_page(page)
        news_links += new_news_links

    news_links = [link for link in news_links if "/noticia/" in link]
    
    #news_links = random.shuffle(news_links)
    news_links = [news_link for news_link in news_links if prepare_link_to_filename(news_link) not in downloaded]

    print("{} links to extract".format(len(news_links)))
    
    for i, news_link in enumerate(tqdm(news_links)):
        try: 
            news_data = extract_text_from_news_link(news_link, driver)
            news_data.to_csv("data/{}/{}.csv".format(team, prepare_link_to_filename(news_link)), index=False)
            #data = pd.concat([data, news_data], sort=True)
        except:
            print("Couldn't get from {}".format(news_link))
        #if i % 5 == 0:
            #print("Saving {} news".format(len(data)))
            #data.to_csv("data/{}.csv".format(team), index=False)

    data = aggregate_all_saved_sections_from_zones_files(team)
    try:
        data["article_time"] = data["date"].apply(lambda x: x.strip().split(" ")[1] if not pd.isnull(x) else x)
        data["article_date"] = data["date"].apply(lambda x: x.strip().split(" ")[0] if not pd.isnull(x) else x)
        data.to_csv("data/{}.csv".format(team), index=False)
    except:
        print("no date")

    
    driver.close()
    return data

def single_page_fetcher(team):
    page = np.random.randint(1, 1500, 1)[0]
    news_from_soccer_club_single(team, page, page + 1)

In [ ]:
team = "flamengo"
for i in range(500):
    try:
        single_page_fetcher(team)
    except:
        print("fail")

  0%|          | 0/2 [00:00<?, ?it/s]/Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
  0%|          | 0/18 [00:00<?, ?it/s]

18 links to extract


  6%|▌         | 1/18 [00:17<04:54, 17.31s/it]/Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
 39%|███▉      | 7/18 [01:42<02:41, 14.71s/it]

Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/apos-aquecimento-na-arena-menino-gremista-ganha-camisa-de-gabigol-e-parece-nao-acreditar.ghtml


  0%|          | 0/20 [00:00<?, ?it/s]

...

Couldn't get from https://globoesporte.globo.com/sportv/programas/troca-de-passes/noticia/pedrinho-comenta-situacao-de-jorge-jesus-flamengo-tinha-que-ter-dado-um-xeque-mate.ghtml


100%|██████████| 19/19 [29:04<00:00, 115.97s/it]

Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/everton-ribeiro-do-flamengo-anuncia-que-esposa-esta-gravida-do-segundo-filo-do-casal.ghtml


fail
fail
fail
fail


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
news = news_from_soccer_club_single("flamengo", start_page=2, end_page=500)

  0%|          | 0/499 [00:00<?, ?it/s]/Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/lgmoneda/miniconda2/envs/ml3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
  0%|          | 0/4599 [00:00<?, ?it/s]

4599 links to extract


  2%|▏         | 78/4599 [28:10<26:03:11, 20.75s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7ea44ccf8>: Failed to establish a new connection: [Errno 61] Connection refused',)': /session/89ae1fb5-a86f-f348-b8c6-b2e692cdc73b/url


Couldn't get from https://globoesporte.globo.com/ce/futebol/noticia/fa-mirim-chora-por-saida-de-jorge-jesus-do-flamengo-outro-tecnico-nao-vai-ser-igual-assista.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/na-europa-flamengo-tenta-dar-o-proximo-passo-pelo-novo-treinador-apos-mapear-o-mercado.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/futebol-internacional/futebol-espanhol/noticia/especulado-no-flamengo-tecnico-unai-emery-assina-com-o-villarreal.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/consenso-e-paciencia-busca-por-tecnico-adia-dialogo-por-futuro-de-diegos-no-flamengo.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/no-primeiro-dia-sem-jorge-jesus-flamengo-tem-conversa-de-braz-com-elenco-e-visita-de-auxiliar-do-mister.ghtml
Couldn't get from https://globoesporte.globo.com/ap/futebol/noticia/flamengo-homenageia-torcedor-de-102-anos-que-venceu-a-co

  2%|▏         | 101/4599 [28:10<12:42:30, 10.17s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7ea457cf8>: Failed to establish a new connection: [Errno 61] Connection refused',)': /session/89ae1fb5-a86f-f348-b8c6-b2e692cdc73b/url


...

 54%|█████▎    | 2462/4599 [29:25<01:57, 18.21it/s]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7cfe05be0>: Failed to establish a new connection: [Errno 61] Connection refused',)': /session/89ae1fb5-a86f-f348-b8c6-b2e692cdc73b/url


 54%|█████▍    | 2486/4599 [29:25<01:23, 25.17it/s]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7e9d54ef0>: Failed to establish a new connection: [Errno 61] Connection refused',)': /session/89ae1fb5-a86f-f348-b8c6-b2e692cdc73b/url


Couldn't get from https://globoesporte.globo.com/programas/esporte-espetacular/noticia/donde-estoy-perguntou-mari-de-abel-a-jesus-veja-o-caminho-do-flamengo-ate-a-final-em-lima.ghtml
Couldn't get from https://globoesporte.globo.com/programas/esporte-espetacular/noticia/futebol-familia-e-a-malta-da-amadora-ee-mostra-raizes-e-historias-de-jorge-jesus.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/contra-o-gremio-flamengo-tem-a-chance-de-igualar-sua-maior-invencibilidade-neste-seculo.ghtml
Couldn't get from https://globoesporte.globo.com/rs/futebol/brasileirao-serie-a/noticia/gremio-x-flamengo-veja-onde-assistir-escalacoes-desfalques-e-arbitragem.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/yuri-cesar-sofre-lesao-no-joelho-e-desfalca-o-flamengo-na-reta-final-do-brasileiro-e-do-carioca-sub-20.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/sem-pablo-mari-e-vitinho-confira-o

 54%|█████▍    | 2503/4599 [29:25<01:02, 33.78it/s]WARNING:urllib3.connectionpool:Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa7cfe05a58>: Failed to establish a new connection: [Errno 61] Connection refused',)': /session/89ae1fb5-a86f-f348-b8c6-b2e692cdc73b/url


Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/zico-evita-comparacoes-entre-o-flamengo-de-1981-e-o-atual-ambos-sao-espetaculares.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/problemas-afastam-arrascaeta-de-meta-contratual-e-flamengo-economiza-quase-r-6-milhoes.ghtml
Couldn't get from https://globoesporte.globo.com/volei/superliga-feminina/noticia/de-volta-a-elite-do-volei-feminino-apos-13-anos-flamengo-e-batido-pelo-minas-na-superliga.ghtml
Couldn't get from https://globoesporte.globo.com/basquete/nbb/noticia/marquinhos-e-balbi-comandam-flamengo-bate-pato-e-vence-a-3a-seguida-no-nbb.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/zico-visita-ninho-do-urubu-sorri-com-jorge-jesus-e-da-a-bencao-a-gabigol.ghtml
Couldn't get from https://globoesporte.globo.com/futebol/times/flamengo/noticia/ingressos-para-a-final-da-libertadores-destinados-a-torcida-do-flamengo-estao-esgotados.ght

In [ ]:
news.head()

In [ ]:
news.shape